# Interaction with the contract

### Web3 instanciation and setup

**NOTE:** Before running this, ensure the blockchain is up by running `deploy.sh`.  

In [21]:
from dotenv import dotenv_values
from functools import wraps
from web3 import Web3
import json

url = "http://127.0.0.1:8545"

# Output from the deploy
contract_address = "0xfcc6285764F77c1671a2DdB177587d893c44EF0D"


env = dotenv_values(".env")
password = env["ACCOUNT_PASSWORD"]

# Also output from deploy
funder_addresses = [
    "0x49aD6eD823fB54A66997Ba26c73a1ebCc90C7b82",
    "0x35b30d66C742b64a8cE690E6824De0aE25C1EDCA",
]
DEFAULT_AMOUNT_FUND = 0.5 # ETH
UNLOCK_DURATION = 300 # Seconds

w3 = Web3(Web3.HTTPProvider(url))

In [22]:
def fund_new_wallet_decorator(func):
    """Wrapper function to automatically send funds from funder adresses to newly created accounts."""

    @wraps(func)
    def wrapper(*args, **kwargs):
        new_wallet_address, new_wallet_private_key = func(*args, **kwargs)

        # Unlock pre-funder wallets
        for address in funder_addresses:
            w3.geth.personal.unlock_account(
                address, password, UNLOCK_DURATION
            )  # Unlock for 5 minutes

        gas_price = w3.eth.gas_price
        gas_limit = w3.eth.estimate_gas(
            {
                "to": new_wallet_address,
                "value": w3.to_wei(DEFAULT_AMOUNT_FUND, "ether"),
                "from": funder_addresses[0],
            }
        )

        for address in funder_addresses:
            transaction = {
                "to": new_wallet_address,
                "value": w3.to_wei(DEFAULT_AMOUNT_FUND, "ether"),
                "gas": gas_limit,
                "gasPrice": gas_price,
                "from": address,
            }
            txn_hash = w3.eth.send_transaction(transaction)
            receipt = w3.eth.wait_for_transaction_receipt(txn_hash)

        return new_wallet_address, new_wallet_private_key

    return wrapper

### Registering an account in the Blockchain

In [23]:
@fund_new_wallet_decorator
def create_new_wallet(password: str):
    """Creates a new wallet in the blockchain"""
    new_wallet = w3.eth.account.create(password)
    new_wallet_address = new_wallet.address
    new_wallet_private_key = new_wallet._private_key.hex()

    return new_wallet_address, new_wallet_private_key


# Usage Example
new_wallet_address, new_wallet_private_key = create_new_wallet("password")

print(f"New Wallet Address: {new_wallet_address}")
print(f"New Wallet Private Key: {new_wallet_private_key}")
print(f"New Wallet Balance: {w3.eth.get_balance(new_wallet_address)}")

New Wallet Address: 0xC4A81375372FF615617f0F1B9786E741E85dBAeF
New Wallet Private Key: 0xe53371fec39b0179137370dc0db90d33d8839830342674d91d7620ea9eff5e9a
New Wallet Balance: 1000000000000000000


### Interacting with the contract

In [24]:
with open("./build/contracts_Voting_sol_Voting.abi", "r") as f:
    contract_abi = json.load(f)
contract = w3.eth.contract(address=contract_address, abi=contract_abi)

# Election and District info

district_name = "Morrinhos"
district_desc = "Município do Estado de Morrinhos"
district_ids = [0]

election_id = 0
election_name = "Eleição Municipal de Morrinhos"
description = "Eleição oficial para prefeitos da melhor cidade da região"

# Creating an election

tx = contract.functions.createElection(election_name, description, [0]).build_transaction(
    {
        "from": new_wallet_address,
        "gas": 2000000,
        "gasPrice": w3.to_wei("20", "gwei"),
        "nonce": w3.eth.get_transaction_count(new_wallet_address),
    }
)

signed_tx = w3.eth.account.sign_transaction(tx, private_key=new_wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# Add district to election

tx = contract.functions.addDistrict(
    election_id, district_name, district_desc
).build_transaction(
    {
        "from": new_wallet_address,
        "gas": 2000000,
        "gasPrice": w3.to_wei("20", "gwei"),
        "nonce": w3.eth.get_transaction_count(new_wallet_address),
    }
)

signed_tx = w3.eth.account.sign_transaction(tx, private_key=new_wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
contract.functions.getDistrictElections(0).call()

[(0,
  'Eleição Municipal de Morrinhos',
  'Eleição oficial para prefeitos da melhor cidade da região',
  '0xC4A81375372FF615617f0F1B9786E741E85dBAeF',
  '0x0000000000000000000000000000000000000000',
  0,
  9999999999,
  False,
  [0],
  1722271316,
  1722271316)]